<h2><center>Step 2: Six New Variables Added to InfoGroup by Matching to External Data Sources</center></h2>

##### 1. Urban Area-Dependent Variables

In [ ]:
The Census Bureau's concept of Urban Area includes two urban categories: the more densely
populated Urbanized Area and the Urban Cluster. See the gazetteer for the details of 
definition. Urban Areas are not defined in terms of any other standard spatial unit. The
borders of an urban area are defined by the density of commuting patterns in the orbit of
urban cores of various population size.

InfoGroup does not include the code for the Urbanized Area or Urban Cluster in which an
establishment may be located. The Bureau does distribute a shapefile for Urban Areas. It
would therefore be possible in theory to locate each establishment's locational coordinates
in an Urban Area or to determine that it is not included in any Urban Area. However, this 
would be 1) an incredibly CPU-intensive process; and 2) probably irrelevant since we are
concerned mostly with InfoGroup establishments in rural areas.

However, because we do have an establishment's census tract code on the InfoGroup record, 
we can determine with just barely imperfect accuracy whether an establishment is
located in a census tract that is itself centered in an Urban Area or non-urban territory.

We have created a geo-reference file starting with shapefiles for Urban Areas and census 
tracts. The centroid location of each census tract was computed and from that data point 
and the coordinate dimensions of each urban area, the 'parental' urban area, if any,
of each census tract was determined. This was an extrememly machine-intensive process itself.

##### rural_outside_UA, UA Code, UA Type

In [ ]:
The 'rural_outside_UA' variable identifies tracts that are not located within a Census Bureau 
Urban Area. More precisely, if the spatial centroid of the InfoGroup establishment's census 
tract is not located within the polygon of coordinates that defines an Urban Area, the 
establishment is considered 'rural' and coded '1'. In the 2017 file, 3,596,102 establishments,
24.5% of the total, were flagged 'rural' by this measure.

For the 'urban' establishments (coded '0' on 'rural_outside_UA') we also take the Census code 
for its 'parental' Urban Area ('UA Code') and the code for the parental urban area's type 
('UA Type'): 'U' = Urbanized Area, 'C' = Urban Cluster.

The accuracy of these three variables is 'just barely imperfect' because a census
tract can overlap multiple urban (or non-urban) areas. It is therefore not necessarily 
true that the urban area pinpointed by the centroid of the census tract is the one in which 
the InfoGroup establishment itself is actually located, although in nearly every case it 
would be.

In [ ]:
Our locally processed geo-reference file 
('/InfoGroup/data/rurality/reference/geographical/points-in-polygons/data/all_tracts.csv')
consists of one record per 2010 census tract, with the following variables:
    'STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC',
    'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry',
    'UA_GEOID10', 'UATYP10', 'rural_tract'
'STATEFP' through 'geometry' are simply taken from the Census Bureau's shapefile. 
'GEOID' is the file's 11-digit census tract identifier. 'UA-GEOID10' and 'UATYP10' are the 
Urban Area identifier and the Urban Area type code taken from the Urban Area shapefile, and 
'rural_tract' is the laboriously computed rurality flag for each census tract renamed to
'rural_outside_UA' in the InfoGroup record to distinguish it from other such indicator
variables to be created in step 3.

Having created this file at an earlier time, adding the last three variables to the InfoGroup
record was simply a matter of a pandas dataframe merge, where 'df' is the InfoGroup dataframe
and 'tract_df' is the dataframe created from 'all_tracts.csv'.

    merged = df.merge(tract_df,how='inner',left_on='Full Census Tract',right_on='GEOID')

##### 2. ERS Measures of Urban Spatial Effect

In [ ]:
The ERS's three measures of urban influence and spatial effect are the Urban Influence codes,
the Urban-Rural Continuum codes, and the Urban-Rural Commuting Area codes. These measures
are applied to the InfoGroup record by simple pandas merges as described below. The source
files are downloaded as Excel spreadsheets and processed into csv text files and finally
read into pandas dataframes.

The ERS has created files of all three codes for a variety of years. The unit of analysis
for the Urban Influence codes and the Rural-Urban Continuum codes is the county. For the
Rural-Urban Commuting Area codes the unit of analysis is the census tract.

##### UI_CODE

In [ ]:
https://www.ers.usda.gov/data-products/:
"The 2013 Urban Influence Codes form a classification scheme that distinguishes metropolitan 
counties by population size of their metro area, and nonmetropolitan counties by size of the 
largest city or town and proximity to metro and micropolitan areas. The standard Office of 
Management and Budget (OMB) metro and nonmetro categories have been subdivided into two 
metro and 10 nonmetro categories, resulting in a 12-part county classification."

See https://www.ers.usda.gov/data-products/urban-influence-codes/.
Urban Influence codes "form a classification scheme that distinguishes metropolitan counties
by population size of their metro area, and nonmetropolitan counties by size of the largest 
city or town and proximity to metro and micropolitan areas. The standard Office of Management 
and Budget (OMB) metro and nonmetro categories have been subdivided into two metro and 10 
nonmetro categories, resulting in a 12-part county classification."

There are separate ERS collections of Urban Influence codes for 1974, 1983, 1993, 2001, 
and 2013. Having chosen a single year of data as appropriate for the particular year or 
years of InfoGroup data, the command to apply UI_CODE to the InfoGroup record, where
'df' is the dataframe of InfoGroup data and 'ui_df' is the dataframe of Urban Influence data,
is:
    merged = df.merge(ui_df,how='inner',left_on='FIPS Code',right_on='FIPS')

##### RUC_CODE

In [ ]:
https://www.ers.usda.gov/data-products/:
"The 2013 Rural-Urban Continuum Codes form a classification scheme that distinguishes 
metropolitan counties by the population size of their metro area, and nonmetropolitan 
counties by degree of urbanization and adjacency to metro areas. The official Office of 
Management and Budget (OMB) metro and nonmetro categories have been subdivided into three
metro and six nonmetro categories. Each county in the U.S. and Puerto Rico is assigned one 
of the 9 codes."

See https://www.ers.usda.gov/data-products/rural-urban-continuum codes/. 
Rural-Urban Continuum codes "form a classification scheme that distinguishes metropolitan 
counties by the population size of their metro area, and nonmetropolitan counties by degree 
of urbanization and adjacency to a metro area. The official Office of Management and Budget 
(OMB) metro and nonmetro categories have been subdivided into three metro and six nonmetro 
categories. Each county in the U.S. is assigned one of the 9 codes."

There are separate ERS collections of Rural-Urban Continuum codes for 1993, 2003, and 2013.
Having first chosen a single year of RUC data, the command to apply RUC_CODE to the InfoGroup 
record, where is:
    merged = df.merge(ruc_df,how='inner',left_on='FIPS Code',right_on='FIPS')

##### RUCA_CODE

In [ ]:
See https://www.ers.usda.gov/data-products/rural-urban-commuting-area-codes/.
Rural-Urban Commuting Area codes "classify U.S. census tracts using measures of population 
density, urbanization, and daily commuting....The classification contains two levels. Whole 
numbers (1-10) delineate metropolitan, micropolitan, small town, and rural commuting areas 
based on the size and direction of the primary (largest) commuting flows. These 10 codes are 
further subdivided based on secondary commuting flows, providing flexibility in combining 
levels to meet varying definitional needs and preferences."

There are separate ERS collections of Rural-Urban Commuting Area codes for 1990, 2000, 
and 2010. The three years of RUCA codes "are not directly comparable because many census 
tracts are reconfigured during each decade. Also, changes to census methodologies 
significantly affected the RUCA classifications."

Having first chosen a single year of RUCA data, the command to apply RUC_CODE 
to the InfoGroup record, where is:
    merged = df.merge(ruca_df,how='inner',left_on='Full Census Tract',right_on='FIPS')        